# DataPipe Typing System

DataPipe typing system is introduced to make the graph of DataPipes more reliable and provide type inference for users. The typing system provide the flexibility for users to determine which level(s) to have type enforcement and risk false positive errors.

In [1]:
from torch.utils.data import IterDataPipe
from typing import Any, Iterator, List, Tuple, TypeVar, Set, Union

T_co = TypeVar('T_co', covariant=True)

In [2]:
# Hide traceback of Error
import functools
ipython = get_ipython()
ipython.showtraceback = functools.partial(ipython.showtraceback, exception_only=True)

## Compile-time
Compile-time typing is enabled by default for now. And it will generate an attribute of `type` for each DataPipe. If there is no type hint specified, the DataPipe is set to a default type `Any`.

### Invalid Typing
- Return type hint of `__iter__` is not `Iterator`

In [3]:
class InvalidDP1(IterDataPipe[int]):
    def __iter__(self) -> str:
        pass

TypeError: Expected 'Iterator' as the return annotation for `__iter__` of InvalidDP1, but found str

- Return type hint of `__iter__` doesn't match or is subtype of the declared type hint

In [4]:
class InvalidDP2(IterDataPipe[int]):
    def __iter__(self) -> Iterator[str]:
        pass

TypeError: Expected return type of '__iter__' is a subtype of int, but found str for InvalidDP2

### Valid Typing

- It's allowed that return type is a subtype of class type annotation

In [5]:
class DP(IterDataPipe[Tuple]):
    def __iter__(self) -> Iterator[Tuple[int, str]]:
        pass

In [6]:
class DP(IterDataPipe):
    def __iter__(self) -> Iterator[int]:
        pass

- Default Typing (Any) with/without return hint for `__iter__`

In [7]:
class DP(IterDataPipe):
    def __iter__(self):
        pass
print(DP.type)
class DP(IterDataPipe):
    def __iter__(self) -> Iterator:
        pass
print(DP.type)
class DP(IterDataPipe):
    def __iter__(self) -> Iterator[T_co]:
        pass
print(DP.type)

typing.Any
typing.Any
typing.Any


- Matched type hints (including equal but not same types)

In [8]:
class DP(IterDataPipe[Tuple[T_co, str]]):
    def __iter__(self) -> Iterator[Tuple[T_co, str]]:
        pass
print(DP.type)

T = TypeVar('T', int, str)  # equals to Union[int, str]
class DP(IterDataPipe[Tuple[T, str]]):
    def __iter__(self) -> Iterator[Tuple[Union[int, str], str]]:
        pass
print(DP.type)

typing.Tuple[+T_co, str]
typing.Tuple[~T, str]


### Attribute `type`
The attribute `type` is added into each DataPipe class.

In [9]:
def print_helper(cls, obj):
    print("DataPipe[{}]\nInstance type: {}"
          .format(cls.type, obj.type))

In [10]:
class DP(IterDataPipe[List[int]]):
    def __iter__(self) -> Iterator[List[int]]:
        pass
print_helper(DP, DP())

DataPipe[typing.List[int]]
Instance type: typing.List[int]


In [11]:
class DP(IterDataPipe[Any]):
    def __iter__(self) -> Iterator[Any]:
        pass
print_helper(DP, DP())

DataPipe[typing.Any]
Instance type: typing.Any


In [12]:
class DP(IterDataPipe[tuple]):
    def __iter__(self) -> Iterator[tuple]:
        pass
print_helper(DP, DP())

DataPipe[tuple]
Instance type: tuple


## Construct-time

Construct-time type checking is enabled by a decorator `construct_time_validation`. Users can opt in by attaching the decorator to `__init__` or `__new__` function, then users can do operations based on the typing of input `DataPipe`(s).

Note: This decorator is only allowed to be attached to `__init__` or `__new__` for now. If there is any objection, we can leverage this limitation to any function.

In [13]:
from torch.utils.data import construct_time_validation

class DP(IterDataPipe):
    @construct_time_validation
    def __init__(self, dp: IterDataPipe[Union[int, tuple]]):
        self.dp = dp

    def __iter__(self):
        for d in self.dp:
            yield d

In [14]:
dp = DP(range(10))

TypeError: Expected argument 'dp' as a IterDataPipe, but found <class 'range'>

- When any input is annotated by `IterDataPipe` with detail typing hints, the `type` of input instance must be a subtype of the hint.

In [15]:
class Temp(IterDataPipe[str]):
    def __iter__(self):
        pass
dp = DP(Temp())

TypeError: Expected type of argument 'dp' as a subtype of hint typing.Union[int, tuple], but found str

- Example of valid input `DataPipe`

In [16]:
class Temp(IterDataPipe[Tuple[int, T_co]]):
    def __iter__(self):
        pass
dp = DP(Temp())

## Runtime

Runtime type checking is enabled by a decorator `runtime_validation`. Users can opt in by attaching the decorator to `__iter__` to check the output data is an instance of subtype of `type` attribute of the DataPipe.

Note: This decorator is only allowed to be attached to `__iter__` for now. It can be extended into `__getitem__` and further `nonblocking` functions.

In [17]:
from torch.utils.data import runtime_validation

class DP(IterDataPipe[Tuple[int, T_co]]):
    def __init__(self, datasource):
        self.ds = datasource
        
    @runtime_validation
    def __iter__(self):
        for d in self.ds:
            yield d

- Raise `RuntimeError` when the data is not of subtype

In [18]:
dp = DP([(1, 1), (2, 2), ('3', 3)])
for d in dp:
    print(d)

(1, 1)
(2, 2)


RuntimeError: Expected an instance of subtype typing.Tuple[int, +T_co], but found ('3', 3)

In [19]:
dp = DP([(1, 1), (2, 2), [3, 3]])
for d in dp:
    print(d)

(1, 1)
(2, 2)


RuntimeError: Expected an instance of subtype typing.Tuple[int, +T_co], but found [3, 3]

- No error will be raised when all data pass the validation

In [20]:
dp = DP([(1, 1), (2, '2'), (3, 3.)])
for d in dp:
    print(d)

(1, 1)
(2, '2')
(3, 3.0)
